## Create an RDS instance


First, we need to create a Security Group allowing incoming connections to port 3306 (the default MySQL port).

https://boto3.readthedocs.io/en/latest/reference/services/ec2.html#EC2.Client.create_security_group
https://boto3.readthedocs.io/en/latest/reference/services/ec2.html#EC2.Client.authorize_security_group_ingress

Then, we launch an RDS MySQL instance.

https://boto3.readthedocs.io/en/latest/reference/services/rds.html#RDS.Client.create_db_instance

**Make sure to use the 'db.t2.micro' instance class to stay within the free tier.**

In [ ]:
import boto3

ec2 = boto3.client('ec2', region_name='eu-west-1')
response = ec2.create_security_group(GroupName='meetupdbsg',Description='SG for ML Meetup')
sgid = response['GroupId']
ec2.authorize_security_group_ingress(GroupId=sgid,IpProtocol='tcp',CidrIp='0.0.0.0/0',FromPort=3306,ToPort=3306)

rds_client = boto3.client('rds', region_name='eu-west-1')
rds_client.create_db_instance(
    DBName='meetupdb', DBInstanceIdentifier='meetuprdsinstance', 
    Engine='mysql', EngineVersion='5.7.11', DBInstanceClass='db.t2.micro', AllocatedStorage=8,
    MasterUsername='julien', MasterUserPassword='julienjulien',
    VpcSecurityGroupIds=[sgid])


Let's vizualize some properties of our instances. Once it's in the 'available' state, we'll be able to connect to it.

In [ ]:
import boto3

rds_client = boto3.client('rds', region_name='eu-west-1')
instances = rds_client.describe_db_instances()
for i in instances['DBInstances']:
    #print i
    print 'Name: ' + i['DBInstanceIdentifier']
    print 'Status: ' + i['DBInstanceStatus']
    print 'Endpoint: ' + i['Endpoint']['Address']
    print 'Port: ' + str(i['Endpoint']['Port'])
    print


Let's connect to our instance, for example with the standard 'mysql' client:

```
$ mysql -h $ENDPOINT -u julien -p
Enter password:
Welcome to the MySQL monitor.  Commands end with ; or \g.
Your MySQL connection id is 16
Server version: 5.7.11-log MySQL Community Server (GPL)

Copyright (c) 2000, 2016, Oracle and/or its affiliates. All rights reserved.

Oracle is a registered trademark of Oracle Corporation and/or its
affiliates. Other names may be trademarks of their respective
owners.

Type 'help;' or '\h' for help. Type '\c' to clear the current input statement.

mysql>
mysql>
mysql> show databases;
+--------------------+
| Database           |
+--------------------+
| information_schema |
| innodb             |
| meetupdb           |
| mysql              |
| performance_schema |
| sys                |
+--------------------+
6 rows in set (0,06 sec)
```




## Delete an RDS instance

When you're done working with your instance, don't forget to delete it. **Be careful, there is no confirmation!**

You can use the describe_db_instances() script above to check that your instance is being deleted.

In [ ]:
import boto3

rds_client = boto3.client('rds', region_name='eu-west-1')
response = rds_client.delete_db_instance(DBInstanceIdentifier='meetuprdsinstance',SkipFinalSnapshot=True)
print response
